In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Finding race years for which the data is being processed

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_file_date}'") 

In [0]:
race_year_list = df_column_to_list(race_results_df, "race_year")


In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

driver_standing_df = race_results_df\
    .groupBy("race_year", "driver_name", "nationality") \
    .agg(sum("points").alias("total_points"), 
         count(when(col("position") == 1, True)).alias("wins"))

In [0]:
# display(driver_standing_df.filter("race_year = 2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = driver_standing_df.withColumn("rank", rank().over(driver_rank_spec))



In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

In [0]:
# overwrite_partision(final_df, 'f1_presentation', 'driver_standings', 'race_year')

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"
merge_delta_data(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, merge_condition, 'race_year')

In [0]:
%sql
SELECT * FROM f1_presentation.driver_standings WHERE race_year = 2021;

In [0]:
%sql
SELECT race_year, COUNT(1)
FROM f1_presentation.driver_standings
GROUP BY race_year
ORDER BY race_year DESC